In [2]:
%%capture
! pip install datasets transformers seqeval

In [3]:
import pandas as pd

## Read Selected Topics courpus data

In [4]:
data=pd.read_pickle("/kaggle/input/earingscallscleaned/motley-fool-data-cleaned.pkl")

In [5]:
data

,date,exchange,ticker,transcript,Call participants,Q&A,quad-date,quad-number
9910,"Feb 18, 2020, 4:30 p.m. ET",NYSE: A,A,good afternoon welcome agilent technologies ' ...,Ankur Dhingra -- Vice President of Investor Re...,:\nOperator\n[Operator Instructions] Your firs...,2020,1
790,"May 21, 2020, 4:30 p.m. ET",NYSE: A,A,good afternoon welcome agilent technologies ' ...,Ankur Dhingra -- Vice President of Investor Re...,:\nOperator\n[Operator Instructions] And our f...,2020,2
10319,"Aug 18, 2020, 4:30 p.m. ET",NYSE: A,A,good afternoon welcome agilent technologies th...,Ankur Dhingra -- Vice President of Investor Re...,:\nOperator\nCertainly. [Operator Instructions...,2020,3
9879,"Nov 23, 2020, 4:30 p.m. ET",NYSE: A,A,"good afternoon , welcome agilent technologies ...",Ankur Dhingra -- Vice President of Investor Re...,:\nOperator\nCertainly. [Operator Instructions...,2020,4
10252,"Feb 16, 2021, 4:30 p.m. ET",NYSE: A,A,good afternoon welcome agilent technologies fi...,Ankur Dhingra -- Vice President of Investor Re...,:\nAnkur Dhingra -- Vice President of Investor...,2021,1
...,...,...,...,...,...,...,...,...
5881,"Oct 27, 2020, 4:15 p.m. ET",NASDAQ: ZYXI,ZYXI,"good afternoon , everyone , welcome zynex thir...","Thomas Sandgaard -- Chairman of the Board, Pre...",:\nOperator\n[Operator Instructions] Our first...,2020,3
9550,"Feb 25, 2021, 4:15 p.m. ET",NASDAQ: ZYXI,ZYXI,"good afternoon , welcome zynex fourth quarter ...","Thomas Sandgaard -- Chairman of the Board, Pre...",":\nThomas Sandgaard -- Chairman of the Board, ...",2020,4
9880,"Apr 29, 2021, 4:15 p.m. ET",NASDAQ: ZYXI,ZYXI,"good afternoon , welcome zynex 2021 first quar...","Thomas Sandgaard -- Chairman of the Board, Pre...",:\nOperator\n[Operator Instructions]. Our firs...,2021,1
17363,"Jul 29, 2021, 4:15 p.m. ET",NASDAQ: ZYXI,ZYXI,"good day , welcome zynex 2021 second quarter c...","Thomas Sandgaard -- Chairman, President & Chie...",:\nOperator\nThank you. [Operator Instructions...,2021,2


## Split the transcript to chunks of 256 tokens then apply textblon to sentiment labeling

In [39]:
import math
from tqdm.notebook import tqdm
splitSize=256
chunks=[]
chunksSentiment=[]
for idx in tqdm(range(len(data["transcript"]))):
    tokens=data["transcript"].iloc[idx].split()
    chunks=chunks+[" ".join(x) for x in [tokens[r*splitSize:(r+1)*splitSize] for r in range(math.ceil(len(tokens)/splitSize))]]


  0%|          | 0/17592 [00:00<?, ?it/s]

In [69]:
%%time
pip install textblob

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.
CPU times: user 196 ms, sys: 83.3 ms, total: 280 ms
Wall time: 12.2 s


In [70]:
from textblob import TextBlob

def get_sentiment(text):
    analysis = TextBlob(text)
    
    # Sentiment polarity ranges from -1 (negative) to 1 (positive)
    # Here, we consider values above 0 as positive and values below 0 as negative.
    if analysis.sentiment.polarity > 0:
        return "Positive"
    else:
        return "Negative"



The sentiment of the text is: Positive


In [71]:
chunksSentiment=[get_sentiment(c) for c in tqdm(chunks)]

  0%|          | 0/161207 [00:00<?, ?it/s]

In [80]:
maps={"Positive":1,"Negative":0}
chunksSentiment=[maps[x] for x in chunksSentiment]

In [81]:
chunks[1]

"ankur thanks everyone joining call today . 'd like start today 's call reminder mark doak , acg group retiring may 1st . mark wife currently enjoying long-planned vacation able join us today , would remiss taking opportunity recognize outstanding accomplishments mark made stellar 38-year career . track record results speak . thank , mark . strong bench agilent already named mark 's successor , padraig mcdonnell . padraig knows business well . 's mark 's staff several years currently runs chemistries supplies division . padraig mark already working transition activities padraig prepares take helm acd business start fiscal third quarter . congratulate mark padraig onto quarterly results . agilent team delivered strong start 2020. first quarter revenues expectations business grew regions markets . total revenues $ 1.36 billion 5.7 % year-over-year reported basis 2.4 % core basis . continued translate top line growth strong bottom line earnings . earnings per share $ 0.81 7 % , high-end g

In [82]:
chunksSentiment[1]

1

## Create Dataset

In [83]:
from datasets import Dataset
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

In [84]:
usedData = pd.DataFrame({"transcript": chunks,"labels":chunksSentiment})

In [85]:
usedData = usedData.sample(frac=1).reset_index(drop=True)

In [86]:
usedData["labels"].value_counts()

1    151325
0      9882
Name: labels, dtype: int64

In [87]:
usedData

,transcript,labels
0,", compared 3.4 % home sales revenue last year ...",1
1,quarter . generated strong sales solid demand ...,1
2,"hello , welcome unum group 3q 2021 earnings co...",1
3,"capital . looking ahead , currently $ 300 mill...",1
4,"march projects delayed , continue awarded new ...",1
...,...,...
161202,"helped maintain margins . moving slide nine , ...",1
161203,"good morning , welcome idexx laboratories firs...",1
161204,non-contact measurement tools . fluke industri...,1
161205,. addition functionality like shared log walle...,1


In [88]:
train,test = train_test_split(usedData,test_size=0.3)
validation,test = train_test_split(test,test_size=0.2)
testCopy=test
tds = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
val= Dataset.from_pandas(validation)
ds = DatasetDict()

ds['train'] = tds
ds['validation'] = val
ds["test"]=test
ds
ds = ds.remove_columns("__index_level_0__")
ds

DatasetDict({
    train: Dataset({
        features: ['transcript', 'labels'],
        num_rows: 112844
    })
    validation: Dataset({
        features: ['transcript', 'labels'],
        num_rows: 38690
    })
    test: Dataset({
        features: ['transcript', 'labels'],
        num_rows: 9673
    })
})

In [89]:
testCopy.to_csv("test.csv")

## Finetune Bert

In [90]:
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer,BertForSequenceClassification
import torch
id2label={1:"postive",0:"negative"}
label2id={"postive":1,"negative":0}
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2,id2label=id2label,label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

## Apply Tokeniztion

In [91]:
def tokenize(examples):
      return tokenizer(examples["transcript"],padding="max_length",max_length=512,return_overflowing_tokens=False,add_special_tokens=True,truncation=True)
   

In [92]:
tokenized_dataset=ds.map(tokenize,batched=True)

  0%|          | 0/113 [00:00<?, ?ba/s]

  0%|          | 0/39 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [93]:
len(tokenized_dataset["train"][2]["input_ids"])

512

## Training Phases

In [94]:
%%capture
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [100]:
import evaluate
import numpy as np
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
def compute_metrics (eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="micro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="micro"))
    results.update(precision_metric.compute(predictions=preds, references = labels, average="micro"))
    return results

In [101]:
from huggingface_hub import notebook_login

notebook_login()

In [102]:
from transformers import  TrainingArguments, Trainer

In [103]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [96]:

training_args = TrainingArguments(
    output_dir = "managersFeedback-V1.0.7",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,logging_steps=1,gradient_accumulation_steps=2,
    push_to_hub=True, # set to true to uploading
)

In [104]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,data_collator=data_collator,compute_metrics=compute_metrics
)

Cloning https://huggingface.co/AhmedTaha012/managersFeedback-V1.0.7 into local empty directory.


In [105]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Recall,Precision
1,0.001300,0.061205,0.980719,0.980719,0.980719


TrainOutput(global_step=7053, training_loss=0.10122421219392819, metrics={'train_runtime': 6916.5299, 'train_samples_per_second': 16.315, 'train_steps_per_second': 1.02, 'total_flos': 2.969050393104384e+16, 'train_loss': 0.10122421219392819, 'epoch': 1.0})

In [106]:
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 0.05570419877767563,
 'eval_f1': 0.9819084048382094,
 'eval_recall': 0.9819084048382094,
 'eval_precision': 0.9819084048382094,
 'eval_runtime': 181.4501,
 'eval_samples_per_second': 53.309,
 'eval_steps_per_second': 6.668,
 'epoch': 1.0}

In [107]:
trainer.push_to_hub() 

Upload file runs/Aug27_10-23-59_a65a41583e15/events.out.tfevents.1693131925.a65a41583e15.28.0:   3%|2         …

To https://huggingface.co/AhmedTaha012/managersFeedback-V1.0.7
   8a710df..3a84b09  main -> main



Upload file runs/Aug27_10-23-59_a65a41583e15/events.out.tfevents.1693139341.a65a41583e15.28.1: 100%|##########…

To https://huggingface.co/AhmedTaha012/managersFeedback-V1.0.7
   3a84b09..ecd4834  main -> main



'https://huggingface.co/AhmedTaha012/managersFeedback-V1.0.7/commit/3a84b099fc250fb142b4352d5c56e52943996e3a'

In [108]:
from transformers import pipeline

pipe = pipeline("text-classification", model="AhmedTaha012/managersFeedback-V1.0.7")

In [127]:
testData=pd.read_csv("/kaggle/working/test.csv")

In [128]:
testData

,Unnamed: 0,transcript,labels
0,95218,good morning welcome cnfinance first quarter 2...,1
1,26869,"good day , thank standing . welcome arconic co...",1
2,22955,costs related acquisition new clients despite ...,1
3,30185,superior execution remaining focused strategic...,1
4,156369,'s losses bankruptcy filing date . deconsolida...,1
...,...,...,...
9668,12096,'s shift quick thoughts previously announced t...,1
9669,44217,"quick follow-up capital structure . mike , com...",1
9670,119117,", targeted increases discretionary spending dr...",1
9671,77085,outside hilltop holdings purchased market valu...,0


In [134]:
import math
splitSize=256
tokens=data["transcript"].iloc[5].split()
chunks=[" ".join(x) for x in [tokens[r*splitSize:(r+1)*splitSize] for r in range(math.ceil(len(tokens)/splitSize))]]

In [135]:
chunks

["good afternoon welcome agilent technologies second quarter earnings conference call . [ operator instructions ] 'd like introduce host today 's conference , ruben dirado , director , investor relations . please go ahead , sir . thank gabriel , welcome everyone agilent 's second quarter conference call fiscal year 2021. mike mcmullen , agilent 's president ceo ; bob mcmahon , agilent 's senior vice president chief financial officer . joining q & bob mike 's comments jacob thaysen , president agilent 's life science applied markets group ; sam raha , president agilent 's diagnostics genomics group ; padraig mcdonnell , president agilent crosslab group . presentation webcast live . news release , investor presentation information supplement today 's discussion , along recording webcast made available website investor.agilent.com . today 's comments mike bob refer non-generally accepted accounting principles financial measures . find directly comparable generally accepted accounting prin

In [136]:
perdiction=[pipe(x)[0]["label"] for x in chunks]
finalpred=max(perdiction,key=perdiction.count)

In [139]:
perdiction

['postive',
 'postive',
 'postive',
 'postive',
 'postive',
 'postive',
 'postive',
 'postive',
 'postive']

In [138]:
finalpred

'postive'

In [118]:
perdiction

['postive']

In [33]:
finalpred

'negative'

In [35]:
perdiction

['negative', 'negative', 'negative', 'negative', 'negative', 'negative']

In [38]:
chunks[-3]

", decline 0.8 ppts compared 2q'21 . net non-operating income 3q'21was twd148 compared net non-operating expenses 2q'21 . difference mainly due increase share gain associates accounted using equity method twd85 million . decrease foreign exchange losses twd62 million increase gain valuation financial assets fair value profit loss twd15 million . profit attributable company 3q'21 expanded 9 % compared 2q'21 . basic weighted average outstanding common shares 727 million shares . compared 3q'20 , total revenue 3q'21 25.9 % . gross margin increased 8.0 ppts 27.3 % . operating expenses increased 17.4 % . operating profit margin increased 8.7 ppts 21.3 % . net non-operating income 3q'21were twd148 million compared net non-operating expenses 3q'20 . difference mainly due increase share gain associates accounted using equity method twd201million decrease foreign exchange losses twd112 million . profit 230.4 % compared 3q'20 . difference mainly due increase operating profit twd810 million . net